# Files used by vasp

既可以做输入也可以做输出的文件:   

|文件|说明|
|:---|:---|
|CHGCAR| |
|WAVECAR| |
|TMPCAR| |

## INCAR文件

**INCAR**是VASP主要的输入文件, 包含关于计算的各种参数, 在不知道参数准确含义的情况下, 应该使用默认值;

## STOPCAR文件

**STOPCAR**可以在VASP的执行期间可以终止程序的运行;  
* `LSTOP = .TRUE.`: 在下一个离子步骤终止程序;    
* `LABORT = .TRUE.`: 在一个电子步骤终止程序, 可能导致WAVECAR和CHGCAR包含非自洽数据;

## OSZICAR文件(stdout)

**OSZICAR**保存关于收敛速度和当前步的信息;

![OSZICAR.png](data/OSZICAR.png)

`N`: 电子迭代的步数;  
`E`: 当前自由能;  
`dE`: 当前自由能与前一次自由能的差值;  
`d eps`: 能带能(本征值)的改变值;  
`ncg `: $H\Psi$的优化步数;  
`rms`: R = H − εS|φ >
`rms(c)`: 输入和输出电荷密度之间的差异;

最后一行表示收敛之后的能量

## POTCAR file

POTCAR文件包含了每个原子物种的赝势;    
组合物种的POTCAR文件, 只需简单的将每个物种的POTCAR串联起来即可;    
`cat ~/pot/Al/POTCAR ~/pot/C/POTCAR ~/pot/H/POTCAR >POTCAR`;  
VASp3.2之后的版本中, 每个POTCAR文件中都包含了原子的基本信息, 如mass, valence, the energy of the reference configuration for which the pseudopotential was created(创建赝势所定的参考态能量), 所以这些信息不必在INCAR文件中设置了;  
POTCAR文件中的原子顺序必须与POSCAR中的原子顺序一致;   
POTCAR文件包含默认的截断能(default energy cutoff: ENMAX and ENMIN), 因此不需要再INCAR文件中设置`ENCUT`; 当然, INCAR文件中的`ENCUT`参数会覆盖POTCAR文件中的默认参数;  
对于混合物种, 会从POTCAR中选取最大的( ENMAX or ENMIN)作为`INCAR`中的默认截断能;

## KPOINTS文件

`vasp.5.2.12`可以省略KPOINTS文件, 而在INCAR文件中设置;

### 明确的键入所有的K点

In [10]:
%%pass
Example file   !注释行
4              !K点的数量
Cartesian      !实空间(C,c)还是倒空间(K,k,reciprocal)
0.0 0.0 0.0 1. !三个坐标分量, 权重
0.0 0.0 0.5 1.
0.0 0.5 0.5 2.
0.5 0.5 0.5 4.
Tetrahedra
1 0.183333333333333
6 1 2 3 4

### 对于能带计算的K点设置

In [12]:
%%pass
k-points along high symmetry lines
40 ! 40 intersections
Line-mode
cart
0 0 0 ! gamma
0 0 1 ! X
0 0 1 ! X
0.5 0 1 ! W
0.5 0 1 ! W
0 0 1 ! gamma

In [ ]:
%%pass
k-points along high symmetry lines
40 ! 40 intersections
Line-mode
rec
0 0 0 ! gamma
0.5 0.5 0 ! X
0.5 0.5 0 ! X
0.5 0.75 0.25 ! W
0.5 0.75 0.25 ! W
0 0 0 ! gamma

该种形式的K点用于计算能带结构

In [13]:
%%pass #hexagonal(六角)structure
k-points along high symmetry lines for hexagonal str.
40
line
rec
0.000 0.000 0.500 ! A
0.000 0.000 0.000 ! Gamma
0.000 0.000 0.000 ! Gamma
0.500 0.000 0.000 ! M
0.500 0.000 0.000 ! M
0.333333 0.333333 0.000 ! K
0.333333 0.333333 0.000 ! K
0.000 0.000 0.000 ! Gamma

### 自动生成K-网

In [56]:
%%pass
%%writefile data/KPOINTS.bak
Automatic mesh   !注释行
0                !0表示自动产生K点
Gamma            !G表示使用M-P方法且以Gamma为中心自动产生K点
4  4  4          !沿各个基矢方向分割各基矢的点数
0. 0. 0.         !对所按网格分割产生的K点进行平移的量

In [58]:
br.showfile('data/KPOINTS.bak')

@data/KPOINTS.bak>>                               
  1: Automatic mesh   !注释行
  2: 0                !0表示自动产生K点
  3: Gamma            !G表示使用M-P方法且以Gamma为中心自动产生K点
  4: 4  4  4          !沿各个基矢方向分割各基矢的点数
  5: 0. 0. 0.         !对所按网格分割产生的K点进行平移的量

> 1. $\alpha=\beta=\gamma=90°$时, k点网格大小k1,k2,k3与晶格常数要尽量满足关系: `k1*a=k2*b=k3*c`;
1. k点越密计算精度越高,成本也越高, 对于k点的需求, 金属>>半导体,绝缘体; 块体>表面;   
1. 对于表面体系, z轴方向有真空层, c较大, k3取1即可;  
若a,b均大于15埃, k点通常设置成Gamma点(1X1X1);
1. k点与晶格常数有关, 当两种体系晶格常数不变时, k点需要一致才有可比性(同一体系不同位置的掺杂,晶格常数不变的吸附体系)

In [15]:
%%pass
%%writefile KPOINTS.bak
Automatic mesh    !注释行
0                 ! number of k-points = 0 ->automatic generation scheme
Auto              ! fully automatic
10                ! length (l)

In [ ]:
%%pass
Automatic generation
0
Cartesian
0.25 0.00 0.00
0.00 0.25 0.00
0.00 0.00 0.25
0.00 0.00 0.00

### hexagonal lattices

### POSCAR文件

有关晶格几何(lattice geometry)和离子位置(ionic positions)的文件;  
对于MD-run, 还可以包含起始速度(starting velocities)和坐标预测校正器(predictor-corrector coordinates);

In [53]:
%%pass
%%writefile data/POSCAR.bak
Cubic BN              !注释行
3.57                  !晶格常数的统一缩放系数, 也会缩放原子坐标
0.0 0.5 0.5           !此三行表示晶格矢量
0.5 0.0 0.5
0.5 0.5 0.0
1 1                   !与POTCAR对应,指明每个原子的个数
Selective dynamics    !可选力度: 是否要固定部分原子的坐标, T为不固定, F为固定
Cartesian
0.00 0.00 0.00 T T F  !F:fix, T:不固定;
0.25 0.25 0.25 F F F
                      !此后可添加有关运行分子动力学的参数(MD-run)

In [52]:
br.showfile('data/POSCAR.bak')

@data/POSCAR.bak>>                                
  1: Cubic BN              !注释行
  2: 3.57                  !晶格常数的统一缩放系数, 也会缩放原子坐标
  3: 0.0 0.5 0.5           !此三行表示晶格矢量
  4: 0.5 0.0 0.5
  5: 0.5 0.5 0.0
  6: 1 1                   !与POTCAR对应,指明每个原子的个数
  7: Selective dynamics    !可选力度: 是否要固定部分原子的坐标, T为不固定, F为固定
  8: Cartesian
  9: 0.00 0.00 0.00 T T F  !F:fix, T:不固定;
 10: 0.25 0.25 0.25 F F F
 11:                       !此后可添加有关运行分子动力学的参数(MD-run)

# The INCAR File

## INCAR文件中最长使用的设置

### 静态计算

In [ ]:
%%pass
ISTART = 0 # startjob: no WAVECAR file
ICHARG = 2 # charge: from atoms

ISTART和ICHARG指明如何构建初始的波函数和电荷密度, 是否读入上一次计算的波函数和电荷密度;  
对于能带结构和电子态密度等性质的计算: ISTART=1, ICHARG=11;  
其他情况: ISTART=0, ICHARG=2;

In [62]:
INIWAV = 1   # random initialization for wf.
NELM = 60    #允许电子自洽迭代的最大步数, 电子结构比较复杂的材料可以适当增加该参数
NELMIN = 2   # minimum of two steps
NELMDL = -5 # no update of charge for 3 steps
EDIFF = 10E-4 # 电子自洽的收敛标准, 默认值是1E5,当连续两次迭代中总能和本征值的变化都小于该值时,电子的自洽迭代将终止

## nelm,nelmin,nelmdl

`nelm`: 规定电子自洽(SC, selfconsistency)的最大步数;  
一般情况下不需要改变默认值, 因为, 如果自洽循环在40步内没有收敛, 它可能根本不会收敛; 在这种情况下, 你应该重新考虑标签`IALGO`,(ALGO),`LDIAG`以及混合参数(mixing-parameters);    

`nelmin`: 规定电子自洽的最少步数, 一般情况下无需更改此设置, 在某些情况下(例如MD或离子松弛), 可以将`nelmin`设置为更大的值(4到8);

`nelmdl`: 非自洽步数;

# Pseudopotentials and PAW potentials supplied with the VASP package

VASP提供一组标准赝势(PP), 我们强烈要求所有VASP用户依赖这组PP或PAW势; 生成这些PP是非常困难和耗时的; 创建一个手头上的(available)的赝势包的主要原因是消除了反复(繁琐)测试的需要; 通过依赖全球用户群, 我们可以(几乎)保证该赝势包对大多数应用程序都有效; 对于PP的生成, 过去是现在仍然是一个棘手, 繁琐, 容易出错且耗时的任务, 并且只有少数几个课题组可以负担得起为手头的每个问题生成新的PP;

VASP提供的势是目前可用的最佳赝势, 但赝势方法已经被更好的电子结构方法所取代, 例如PAW方法; 因此, 与VASP一起分发的超软赝势的开发已经结束(the ultra soft pseudopotentials), 我们强烈建议使用VASP-PAW包中现在提供的PAW数据集;

## Ultrasoft pseudopotentials supplied with the VASP package

超软赝势已被PAW赝势取代:    
This section of the manual has not been updated in a long time. We strongly recommend the use of the VASP-PAW potentials described in Sec. 10.2. The use of ultrasoft pseudopotentials is at your own risk and the potentials are no longer maintained or updated.

## The PAW potentials

PAW赝势比US超软赝势更准确:  
* PAW的核半径(core radii)小于US的核半径;
* PAW使用核心区域的所有节点重建了精确的价波函数;
* 对于包含C,N,O元素的计算, PAW并不比US耗时;

对于某些元素, 存在不同版本的PAW赝势:  
标准赝势的软硬程度:
* `..`: 标准版本;    
* `_h`: 比标准赝势更硬, 需要一个更大的截断能;   
* `_s`: 比标准赝势更软; 

把内壳层中的某些电子当做价电子处理:    
* `_pv`: 把$p$电子(半芯态,semi-core)作为价电子处理;  
    * `V_pv`: $3p$电子作为价电子;  
* `_d`: 把$d$电子作为价电子态;
    * `Ga_d`:$3d$电子作为价电子;
* `_sv`: 把$s和p$电子同时作为价电子;
    * `V_sv`: $3s$和$3p$同时作为价电子;   

In [23]:
from pymatgen import Element
V = Element("V")
Ga= Element("Ga")
print("V: ", V.full_electronic_structure[3:])
print("Ga: ", Ga.full_electronic_structure[3:])

V:  [(3, 's', 2), (3, 'p', 6), (3, 'd', 3), (4, 's', 2)]
Ga:  [(3, 's', 2), (3, 'p', 6), (3, 'd', 10), (4, 's', 2), (4, 'p', 1)]


### Recommended PAW potentials for DFT calculations using vasp.5.2

如果化合物中存在具有短键的二聚体(O$ _2$, CO, N$ _2$, F$ _2$, P$ _2$, S$ _2$, Cl$ _2$), 建议使用`_h`赝势(C_h, O_h, N_h, F_h, P_h, S_h, Cl_h);

对应H的赝势而言, 可以包含不同的价电子数(valance); [其他元素的能量和价电子数](http://cmp.xmu.edu.cn/vasp/Recommended_PAW_potentials_DFT_calculations_using_vasp_5_2.html)

| Element | ENMAX (eV) | valency|
|:--|:--|:--|
| H .25| 250| 0.2500|
| H .33| 250| 0.3300|
| H .42| 250| 0.4200|
| H .50| 250| 0.5000|
| H .58| 250| 0.5800|
| H .66| 250| 0.6600|
| H .75| 250| 0.7500|
| H 1.25| 250| 1.2500|
| H 1.33| 250| 1.3300|
| H 1.50| 250| 1.5000|
| H 1.66| 250| 1.6600|
| H 1.75| 250| 1.7500|

### Recommended GW PAW potentials for vasp.5.2

一般情况下, GW赝势比DFT标准赝势的性能好, 尤其是在处理激发态性能上更准确;

如果化合物中存在具有短键的二聚体(O $ _2 $, CO, N $ _2 $, F $ _2 $, P $ _2 $, S $ _2 $, Cl $ _2 $), 我们建议使用_h电位: C_GW_h, O_GW_h, N_GW_h, F_GW_h;     
[GW赝势一览表](http://cmp.xmu.edu.cn/vasp/Recommended_GW_PAW_potentials_vasp_5_2.html)

### 1st row elements

|||||||||||||
|:--|:---|:---|:---|:---|:---|:---|:---|:---|:---|:--|:---|
|B_h| 700| C_h| 700| N_h| 700| O_h| 700| F_h| 700|  | |
|**B**  | 318| C  | 400| N  | 400| O  | 400| F  | 400| Ne| 343|
|B_s| 250| C_s| 273| N_s| 250| O_s| 250| F_s| 250|  | |

对于第一行元素来说:    
大多数情况下, 使用标准赝势;   
对于HF或者杂化类型的计算, 可以使用`标准赝势`, `标准_GW`, 和`标准_硬`, 不能使用`标准_软`赝势;

### Alkali and alkali-earth elements (simple metals)

|||||
|:|:|:|:|
| H    | 250|      |   |
| H_h  | 700|      |   |
| Li   | 140| Be   | 300 |
| Li_sv| 500| Be_sv| 308 |
| Na   | 100| Mg   | 210 |
| Na_pv| 260| Mg_pv| 400 |
| Na_sv| 700|      |   |
| K_pv | 120| Ca_pv| 120 |
| K_sv | 260| Ca_sv| 270 |
| Rb_pv| 120|      |   |
| Rb_sv| 220| Sr_sv| 230 |
| Cs_sv| 220| Ba_sv| 190 |


对于碱金属和碱土金属元素来说:  
使用`_sv`赝势可以提高他们的电子转移性(transferability);  

这些元素通常难以赝势化(pseudize):    
与强电负性元素结合时会造成比通常大的误差;  
目前版本非常精确, 可以提供非常可靠的描述;  

### $ d$-elements

X_pv??X:   
* 早期过渡金属需要使用X_pv, 但是后期过渡金属尤其是贵金属可以冻结$p$态;    
* $3d$元素: Ti,V,Cr使用标准赝势;    
* $4d$元素: 最容易出现问题, Tc_pv之前的元素都用`_pv`;
* $5d$元素: 从Hf开始可以使用标准赝势;

对于HF和杂化类型的计算, 无论何时都要使用`_pv`(或`_sv`)的的赝势

### $ p$-elements, including first row

使用标准赝势

### $ f$-elements

Due to self-interaction errors, $ f$-electrons are not handled well by presently available density functionals